In [51]:
from hashlib import sha256
import json
import time


#block

class block:
    def __init__(self, index, transaction, timestample, prev_hash):
        self.index = index
        self.transaction = transaction
        self.timestample = timestample
        self.prev_hash = prev_hash
        
    def compute_hash(self):
    
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

class transaction:
    def __init__(self, SN, from_key, to_key, validation):
        self.SN = SN
        self.from_key = from_key
        self.to_key = to_key
        self.validation = validation    


chain = dict()
chain['b0'] = block(0,transaction('sis',"gene","n",True),0,0)



def send(from_key, to_key, SN):                   #mandar a transação primeiro para o bloco depois para a chain
    if SN in from_key.parts:
        from_key.parts.pop(from_key.parts.index(SN))
        to_key.parts.append(SN)
        print('part sent')
        
        key = 'b{}'.format(len(chain))
        chain[key] = block(len(chain),transaction(SN, from_key, to_key, True),'time',hash(str(chain['b{}'.format(len(chain)-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)-1)].transaction.SN)))        
        #para recuperar os dados cache["ab123"].SN/from_key...
        
    
    elif SN not in from_key.parts:
        print('Part not in your wallet')
    
    
class wallet:
    def __init__(self,parts,public_key,private_key):
        self.parts = parts
        self.public_key = public_key
        self.private_key = private_key

    import random
    import string
    
    def new_wallet(username):    # 
        username = wallet([],(''.join(random.choices(string.ascii_uppercase + string.digits, k=8))),123)
        
        #criar sistema de explorer
        #ex: hist_part(SN)


In [52]:
a = wallet([123],'a','a1')
b = wallet([],'b','b1')
c = wallet([],'c','c1')

In [53]:
send(a,b,123)
chain

part sent


{'b0': <__main__.block at 0x7f5160e18400>,
 'b1': <__main__.block at 0x7f5160e809e8>}

In [54]:
send(b,c,123)
chain

part sent


{'b0': <__main__.block at 0x7f5160e18400>,
 'b1': <__main__.block at 0x7f5160e809e8>,
 'b2': <__main__.block at 0x7f5160e80cf8>}

In [56]:
chain['b2'].transaction.from_key.public_key

'b'

In [72]:
hash((chain['b{}'.format(len(chain)-1)].transaction.from_key)+(chain['b{}'.format(len(chain)-1)].transaction.SN))

5535373011226918915